# WSI 2022L
# Laboratorium 3 (Dwuosobowe gry deterministyczne)
# Michał Brus, 299106

## Czysty algorytm minimax z obcięciem alfa/beta

In [ ]:
def alpha_beta(position, depth, alpha, beta, max_player):
    if depth == 0 or position == 'terminal':
        return evaluate(position)
    if current_player is max_player:
        max_eval = -math.inf
        for child in children(position):
            eval = alpha_beta(child, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for child in children(position):
            eval = alpha_beta(child, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval


## Algorytm minimax z obcięciem alfa/beta (dostosowany do gry ConnectFour)

In [ ]:
import math

import numpy as np


def evaluate(game_state):
    return game_state.get_heuristic()


def get_winner(game_state):
    return game_state.get_winner()


def current_player(game_state):
    return game_state.get_current_player()


def is_finished(game_state):
    return game_state.is_finished()

def alpha_beta(game_state, depth, alpha, beta, maximising_player):
    rng = np.random.default_rng()
    if depth == 0 or is_finished(game_state):
        if is_finished(game_state):
            winner = get_winner(game_state)
            if winner is current_player(game_state):
                return 10e6, None
            elif winner is not current_player(game_state):
                return -10e6, None
            else:
                return 0, None
        else:
            return evaluate(game_state), None
    if maximising_player:
        max_eval = -math.inf
        possible_moves = game_state.get_moves()
        rng.shuffle(possible_moves)
        best_move = rng.choice(possible_moves)
        for move in possible_moves:
            state = game_state.make_move(move)
            eval = alpha_beta(state, depth - 1, alpha, beta, False)[0]
            if eval > max_eval:
                max_eval = eval
                best_move = move
            elif max_eval == eval:
                best_move = rng.choice([best_move, move])
            else:
                best_move = rng.choice(possible_moves)
            # max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break

        return max_eval, best_move
    else:
        min_eval = math.inf
        possible_moves = game_state.get_moves()
        rng.shuffle(possible_moves)
        best_move = rng.choice(possible_moves)
        for move in possible_moves:
            state = game_state.make_move(move)
            eval = alpha_beta(state, depth - 1, alpha, beta, True)[0]
            if eval < min_eval:
                min_eval = eval
                best_move = move
            elif min_eval == eval:
                best_move = rng.choice([best_move, move])
            else:
                best_move = rng.choice(possible_moves)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move


### Do oceny stanu planszy służy funkcja heurysytczna dodana do kodu gry w pliku: games_examples/two_player_games/games/connect_four.py

Funkcja get_heuristic() korzysta z funkcji pomocniczej _heuristic()

## Uruchomienie gry

In [ ]:
import c4_alfa_beta as ai
import math

from games_examples.two_player_games.player import Player
from games_examples.two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState


def run(depth_1, depth_2, with_print, interactive):
    p1 = Player('a')
    p2 = Player('b')
    game = ConnectFour(first_player=p1, second_player=p2)
    a_points = b_points = 0
    print(str(game))
    while not game.is_finished():
        game_state = game.state
        a_points, a_move = ai.alpha_beta(game_state, depth_1, -math.inf, math.inf, True)
        if with_print:
            print(f"Wartość ruchu a: {a_points} \n")
        game.make_move(a_move)
        if with_print:
            print(str(game))
        if interactive:
            b_move = ConnectFourMove(int(input("Podaj nr kolumny:")))
        else:
            b_points, b_move = ai.alpha_beta(game_state, depth_2, -math.inf, math.inf, False)
            if with_print:
                print(f"Wartość ruchu b: {b_points} \n")
        game.make_move(b_move)
        if with_print:
            print(str(game))
    if game.get_winner() == p1:
        if with_print:
            print("\nWygrywa a!")
        return 'p1'
    elif game.get_winner() == p2:
        if with_print:
            print("\nWygrywa b!")
        return 'p2'
    else:
        if with_print:
            print("\nRemis!")
        return 'r'

Gra może zostać uruchomiona w trybie interaktywnym, gdzie gracz gra z AI lub dwa AI przeciwko sobie. Można też włączyć opcję wyświetlania wyników na konsoli (opcja print).

## Przykładowe uruchomienie z opcją print dla depth_1 = 1, depth_2 = 2

In [ ]:
run(1, 1, True, False)

## Testy i statystyki

In [ ]:
def make_statistics(depth_1, depth_2, to_print):
    runs_number = 30
    sum_a = sum_b = sum_r = 0
    for i in range(runs_number):
        winner = run(depth_1, depth_2, False, False)
        if winner == 'p1':
            sum_a += 1
        elif winner == 'p2':
            sum_b += 1
        else:
            sum_r += 1
    if to_print:
        print(
        f"Wygrane a: {sum_a / runs_number * 100}, wygrane b:{sum_b / runs_number * 100}, remisy: {sum_r / runs_number * 100} w procentach przypadków")

    return 100 * sum_a / runs_number, 100 * sum_b / runs_number, 100 * sum_r / runs_number

Funkcja ta uruchamia 30 gier, aby sprawdzić, który gracz wygrywa procentowo w grach o podanych parametrach (depth_1, depth_2).

In [ ]:
make_statistics(4, 1, False)

In [45]:
make_statistics(4, 1, False)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

KeyboardInterrupt: 

In [ ]:
data = []
for i in range(5):
    for j in range (5):
        data[i][j] = make_statistics(i, j, False)
